In [48]:
ac_labels=  ["safe driving",
"texting - right",
"talking on the phone - right",
"texting - left",
"talking on the phone - left",
"operating the radio",
"drinking",
"reaching behind",
"hair and makeup",
"talking to passenger"]

In [49]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import numpy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K,regularizers
from tensorflow.keras.optimizers import Adam


In [50]:
from tensorflow.keras.models import load_model
model = load_model('vgg_model_finetuned.h5')

In [51]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [52]:
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

In [53]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(ac_labels)

In [54]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions

img_path = 'images (2).jpg'



In [55]:
def preprocess(img):
    #imgi = tf.keras.preprocessing.image.load_img( img,target_size=(224,224))
    x = tf.keras.preprocessing.image.img_to_array(imgi).astype('float32')/255 - 0.5
    x = np.expand_dims(x, axis=0)
    return x

def predict(x):
    
    pred = model.predict(x)
    pred = np.argmax(pred)
    
    return ac_labels[pred]

In [56]:
INPUT_VIDEO_FILE = "C:/Users/user/Desktop/Datasets/input_video.mp4"
OUTPUT_VIDEO_FILE = "C:/Users/user/Desktop/Datasets/output_video1.avi"

vs = cv2.VideoCapture(INPUT_VIDEO_FILE)
vs.set(cv2.CAP_PROP_POS_FRAMES, 20)

writer = None
(W, H) = (None, None)
while True:
    grabbed, frame = vs.read()
    if not grabbed:
        break
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    output = frame.copy()
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224,224))
    frame = np.expand_dims(frame,axis=0).astype('float32')/255 - 0.5
    
    preds = predict(frame)

    text = "activity: {}".format(preds)
    if(preds == 'safe driving'):
        cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 3)
    else:
        cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 255), 3)
        
    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(OUTPUT_VIDEO_FILE, fourcc, 30, (W, H), True)
        
    writer.write(output)
    
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
print("[INFO] cleaning up...")
vs.release()
cv2.destroyAllWindows()


[INFO] cleaning up...
